##### -*- coding: utf8 -*-
### Information
- Date : 2024-03-18 11:00
- Author : Okrie
- Description :   Airplane Price Information Test
- Version : 0.1

In [15]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
import pandas as pd
import requests as req

Naver - https://flight.naver.com/flights/

#### Naver Info     
- seoul all , bk all
- round trip
- depart date = 2024.04.02
- arrival date = 2024.04.05
- adult = 2
- children = 2 ( 4, 10 )      
    
```code
https://flight.naver.com/flights/    
international/                  international    
ICN-BKK-20240402/               departure-arrival-depature_date
BKK-ICN-20240405                departure-arrival-arrival_date
?adults=2                       adult_count      
&child=2                        child_count    
&cabinclass=economy             carbin_class          
&children=1                     children_count         
&infant=1                       infant_count    OPTIONAL     
&isDirect=true                  직항=true OPTIONAL
fareType=Y                      Y 일반, P 프리미엄 일반, C 비지니스, F 일등석
```   
https://flight.naver.com/flights/international/ICN-BKK-20240402/BKK-ICN-20240405?adult=2&child=2&infant=1&isDirect=true&fareType=Y

In [2]:
departure = "ICN"
arrival = "BKK"

departure_date = "2024-04-02"
arrival_date = "2024-04-05"

adults_count = 1
children_count = 0
infant_count = 0

cabinclass = "economy"
direct = True


In [3]:
def convertDateTouseUrl(date: str):
    return date.replace("-", "")

In [4]:
def hasChildrenCheck(children):
    if children > 0:
        return f'&child={children}'
    return ''

def hasInfantCheck(infant):
    if infant > 0:
        return f'&infant={infant}'
    return ''

In [5]:
def isDirect(direct):
    return f'&isDirect=true' if direct else ''

In [6]:
def isFareType(carbinClass):
    if carbinClass == "economy": return "Y"
    if carbinClass == "premium economy": return "P"
    if carbinClass == "business": return "C"
    if carbinClass == "first": return "F"

In [132]:
url = f'https://flight.naver.com/flights/international/'
url += f"{departure.upper()}-{arrival.upper()}-{convertDateTouseUrl(departure_date)}/"
url += f"{arrival.upper()}-{departure.upper()}-{convertDateTouseUrl(arrival_date)}"
url += f'?adults={adults_count}'
url += f"{hasChildrenCheck(children_count)}"
url += f"{hasInfantCheck(infant_count)}"
url += f'{isDirect(direct)}'
url += f'&fareType={isFareType(cabinclass)}'

In [272]:
# 2024-03-21
# API 수집으로 대체

# print(url)

In [271]:
# 2024-03-21
# API 수집으로 대체

# driver = webdriver.Chrome()
# driver.get(url)
# driver.implicitly_wait(30)

In [270]:
# 2024-03-21
# API 수집으로 대체

# driver.delete_all_cookies()
# html = driver.page_source
# soup = BeautifulSoup(html)

concurrent_ConcurrentItemContainer__2lQVG

In [269]:
# 2024-03-21
# API 수집으로 대체

# naver = soup.select('div.concurrent_inner__iqfJr')

In [12]:
columns = [
    'site',
    'dp_departure_airline',
    'dp_departure_img',
    'dp_departure_time',
    'dp_arrival_time',
    'dp_duration',
    'dp_departure_airport',
    'dp_arrival_airport',
    'dp_transit',
    'entry_departure_airline',
    'entry_departure_img',
    'entry_departure_time',
    'entry_arrival_time',
    'entry_duration',
    'entry_departure_airport',
    'entry_arrival_airport',
    'entry_transit',
    'ticket_info',
    'ticket_price',
    'link',
]

df = pd.DataFrame(columns=columns)

In [265]:
# naver

In [267]:
# 2024-03-21
# API 수집으로 대체

# for index, value in enumerate(naver):
#     print(index)
#     # 한 종류의 항공사일 경우
#     if len(value.select('img.airline_logo__3himS'))/2 != 1.0:
#         print(value.select_one('img.airline_logo__3himS').attrs['alt'])
#         print(value.select_one('img.airline_logo__3himS').attrs['src'])
        
#         print(value.select('span.route_airport__3VT7M')[0].b.get_text())
#         print(value.select('span.route_airport__3VT7M')[1].b.get_text())
#         print(value.select('span.route_airport__3VT7M')[0].i.get_text())
#         print(value.select('span.route_airport__3VT7M')[1].i.get_text())
        
#         print(value.select('i.route_info__1RhUH')[0].get_text())
        
#         print(value.select('span.route_airport__3VT7M')[2].b.get_text())
#         print(value.select('span.route_airport__3VT7M')[3].b.get_text())
#         print(value.select('span.route_airport__3VT7M')[2].i.get_text())
#         print(value.select('span.route_airport__3VT7M')[3].i.get_text())
        
#         print(value.select('i.route_info__1RhUH')[1].get_text())
        
#     # 오갈때 항공사 다른 경우
#     else:
#         print(value.select('img.airline_logo__3himS')[0].attrs['alt'])
#         print(value.select('img.airline_logo__3himS')[0].attrs['src'])
        
#         print(value.select('img.airline_logo__3himS')[1].attrs['alt'])
#         print(value.select('img.airline_logo__3himS')[1].attrs['src'])


In [54]:
req_url = "https://airline-api.naver.com/graphql"

In [107]:
adults = f'{adults_count if adults_count > 0 else 0}'
child = f'{children_count if children_count > 0 else 0}'
infant = f'{infant_count if infant_count > 0 else 0}'

In [147]:
def firstGetKey():
    query = 'query getInternationalList($trip: String!, $itinerary: [InternationalList_itinerary]!, $adult: Int = '+adults+'$child: Int = '+child+', $infant: Int = '+infant+', $fareType: String!, $where: String = "pc", $isDirect: Boolean = false, $stayLength: String, $galileoKey: String, $galileoFlag: Boolean = true, $travelBizKey: String, $travelBizFlag: Boolean = true) {        internationalList(            input: {trip: $trip, itinerary: $itinerary, person: {adult: $adult, child: $child, infant: $infant}, fareType: $fareType, where: $where, isDirect: $isDirect, stayLength: $stayLength, galileoKey: $galileoKey, galileoFlag: $galileoFlag, travelBizKey: $travelBizKey, travelBizFlag: $travelBizFlag}        ) {            galileoKey            galileoFlag            travelBizKey            travelBizFlag            totalResCnt            resCnt            results {            airlines            airports            fareTypes            schedules            fares            errors            carbonEmissionAverage {                directFlightCarbonEmissionItineraryAverage                directFlightCarbonEmissionAverage            }           }        }    }'
    
    headers = {
        "Content-Type": "application/json",
        "Referer": url,
    } 

    payload_first = {
        'operationName': "getInternationalList",
        "query": query,
        'variables': {
            'adult': int(adults),
            'child': int(child),
            'fareType': "Y",
            'galileoFlag': True,
            'galileoKey': "",
            'infant': int(infant),
            'isDirect': True,
            'itinerary': [
                {'departureAirport': departure.upper(), 'arrivalAirport': arrival.upper(), 'departureDate': convertDateTouseUrl(departure_date)},
                {'departureAirport': arrival.upper(), 'arrivalAirport': departure.upper(), 'departureDate': convertDateTouseUrl(arrival_date)}
            ],
            'stayLength': "",
            'travelBizFlag': True,
            'travelBizKey': "",
            'trip': "RT",
            'where': "pc",
        }
    }

    response = req.post(url=req_url, headers=headers, json=payload_first)
    return response

In [146]:
def getAirInfo(galileokey, travelbizkey):
    query = 'query getInternationalList($trip: String!, $itinerary: [InternationalList_itinerary]!, $adult: Int = '+adults+'$child: Int = '+child+', $infant: Int = '+infant+', $fareType: String!, $where: String = "pc", $isDirect: Boolean = false, $stayLength: String, $galileoKey: String, $galileoFlag: Boolean = true, $travelBizKey: String, $travelBizFlag: Boolean = true) {        internationalList(            input: {trip: $trip, itinerary: $itinerary, person: {adult: $adult, child: $child, infant: $infant}, fareType: $fareType, where: $where, isDirect: $isDirect, stayLength: $stayLength, galileoKey: $galileoKey, galileoFlag: $galileoFlag, travelBizKey: $travelBizKey, travelBizFlag: $travelBizFlag}        ) {            galileoKey            galileoFlag            travelBizKey            travelBizFlag            totalResCnt            resCnt            results {            airlines            airports            fareTypes            schedules            fares            errors            carbonEmissionAverage {                directFlightCarbonEmissionItineraryAverage                directFlightCarbonEmissionAverage            }           }        }    }'

    headers = {
        "Content-Type": "application/json",
        "Referer": url,
    } 

    payload_first = {
        'operationName': "getInternationalList",
        "query": query,
        'variables': {
            'adult': int(adults),
            'child': int(child),
            'fareType': "Y",
            'galileoFlag': True,
            'galileoKey': galileokey,
            'infant': int(infant),
            'isDirect': True,
            'itinerary': [
                {'departureAirport': departure.upper(), 'arrivalAirport': arrival.upper(), 'departureDate': convertDateTouseUrl(departure_date)},
                {'departureAirport': arrival.upper(), 'arrivalAirport': departure.upper(), 'departureDate': convertDateTouseUrl(arrival_date)}
            ],
            'stayLength': "",
            'travelBizFlag': True,
            'travelBizKey': travelbizkey,
            'trip': "RT",
            'where': "pc",
        }
    }

    response = req.post(url=req_url, headers=headers, json=payload_first)
    return response

In [345]:
resp = firstGetKey().json()
# 데이터 확인
# print(resp)

In [346]:
galileo_key = resp['data']['internationalList']['galileoKey']
biz_key = resp['data']['internationalList']['travelBizKey']

In [347]:
res = getAirInfo(galileokey=galileo_key, travelbizkey=biz_key)

In [348]:
tmp = res.json()

In [349]:
tot = tmp['data']['internationalList']['totalResCnt']
tmp['data']['internationalList']['results']['schedules'][0]
fares = tmp['data']['internationalList']['results']['fares']

In [350]:
# 출국
for i, v in tmp['data']['internationalList']['results']['schedules'][0].items():
    print(\
        i,
        # v['detail'][0],
        v['detail'][0]['sa'],
        v['detail'][0]['ea'],
        v['detail'][0]['av']+v['detail'][0]['fn'],
        v['detail'][0]['sdt'][:8] + " " + v['detail'][0]['sdt'][8:10] + ":" + v['detail'][0]['sdt'][10:12],
        v['detail'][0]['jt'][:2] + ":" + v['detail'][0]['jt'][2:]
    )

print("-"*100)

# 입국
for i, v in tmp['data']['internationalList']['results']['schedules'][1].items():
    print(\
        i,
        # v['detail'][0],
        v['detail'][0]['sa'],
        v['detail'][0]['ea'],
        v['detail'][0]['av']+v['detail'][0]['fn'],
        v['detail'][0]['sdt'][:8] + " " + v['detail'][0]['sdt'][8:10] + ":" + v['detail'][0]['sdt'][10:12],
        v['detail'][0]['jt'][:2] + ":" + v['detail'][0]['jt'][2:]
    )


20240402ICNBKKLJ0001 ICN BKK LJ0001 20240402 19:55 05:40
20240402ICNBKKZE0511 ICN BKK ZE0511 20240402 17:30 05:50
20240402ICNBKKTW0101 ICN BKK TW0101 20240402 20:05 05:50
20240402ICNBKK7C2203 ICN BKK 7C2203 20240402 20:05 05:54
20240402ICNBKK7C2205 ICN BKK 7C2205 20240402 20:40 05:55
20240402ICNBKKBX0747 ICN BKK BX0747 20240402 17:15 06:05
20240402ICNBKKTG0657 ICN BKK TG0657 20240402 10:20 05:50
20240402ICNBKKTG0659 ICN BKK TG0659 20240402 09:35 05:50
20240402ICNBKKTG0653 ICN BKK TG0653 20240402 17:30 05:50
20240402ICNBKKOZ0741 ICN BKK OZ0741 20240402 19:30 05:40
20240402ICNBKKOZ0743 ICN BKK OZ0743 20240402 20:40 06:00
20240402ICNBKKOZ6737 ICN BKK OZ6737 20240402 10:20 05:50
20240402ICNBKKOZ6761 ICN BKK OZ6761 20240402 09:35 05:50
20240402ICNBKKOZ6763 ICN BKK OZ6763 20240402 17:30 05:50
20240402ICNBKKKE0657 ICN BKK KE0657 20240402 09:30 05:45
20240402ICNBKKKE0659 ICN BKK KE0659 20240402 19:45 05:50
20240402ICNBKKKE0651 ICN BKK KE0651 20240402 18:05 05:40
-------------------------------

In [353]:
d = {}
for i, v in fares.items():
    # print(i, '\t', v['fare']['A01'][0]['Adult'])
    # print(v['fare'])
    break

&selectedFlight=1:20240402ICNBKKXJ0703:A01:HK:XJ,2:20240405BKKICNXJ0702:A01:HK:XJ&selectType=concurrent

In [176]:
clickedUrl = url + f'&selectedFlight=1:20240402ICNBKKLJ0001:A01:HK:XJ,2:20240405BKKICNXJ0702:A01:HK:XJ&selectType=concurrent'

In [352]:
clickedUrl

'https://flight.naver.com/flights/international/ICN-BKK-20240402/BKK-ICN-20240405?adults=1&isDirect=true&fareType=Y&selectedFlight=1:20240402ICNBKKLJ0001:A01:HK:XJ,2:20240405BKKICNXJ0702:A01:HK:XJ&selectType=concurrent'

In [273]:
site = "naver"

for index, value in enumerate(naver):
    one_data = {}
    
    # 출국
    dp_departure_airline = value.select('div')[0].span.getText()
    
    dp_departure_img = f"https://{value.select('div.LegLogo_legImage__MmY0Z')[0].img.attrs['src'][2:]}"
    
    dp_departure_time = value.select('div.LegInfo_routePartialDepart__NzEwY')[0].span.get_text()
    dp_arrival_time = value.select('div.LegInfo_routePartialArrive__Y2U1N')[0].span.get_text()
    
    dp_duration = value.select('div.LegInfo_stopsContainer__NWIyN')[0].span.get_text()
    
    dp_departure_airport = value.select('div.LegInfo_routePartialDepart__NzEwY > span:nth-child(2)')[0].span.get_text()
    dp_arrival_airport = value.select('div.LegInfo_routePartialArrive__Y2U1N > span:nth-child(2)')[0].span.get_text()
    
    dp_transit = value.select('div.LegInfo_stopsLabelContainer__MmM0Z')[0].span.get_text()
    
    # 입국
    entry_departure_airline = value.select('div')[1].span.getText()
    
    entry_departure_img  = f"https://{value.select('div.LegLogo_legImage__MmY0Z')[1].img.attrs['src'][2:]}"
    
    entry_departure_time = value.select('div.LegInfo_routePartialDepart__NzEwY')[1].span.get_text()
    entry_arrival_time = value.select('div.LegInfo_routePartialArrive__Y2U1N')[1].span.get_text()
    
    entry_duration = value.select('div.LegInfo_stopsContainer__NWIyN')[1].span.get_text()
    
    entry_departure_airport = value.select('div.LegInfo_routePartialDepart__NzEwY > span:nth-child(2)')[1].span.get_text()
    entry_arrival_airport = value.select('div.LegInfo_routePartialArrive__Y2U1N > span:nth-child(2)')[1].span.get_text()
    
    entry_transit = value.select('div.LegInfo_stopsLabelContainer__MmM0Z')[1].span.get_text()
    
    # Ticket 정보 및 가격
    ticket_info = value.select('div.TicketStub_horizontalStubContainer__YjM5M')[0].span.get_text()
    ticket_price = value.select('div.Price_mainPriceContainer__MDM3O')[0].span.get_text()
    
    # 데이터 넣기전 임시
    link = "naver.com"
    
    one_data = {
        'site' : site,
        'dp_departure_airline' : dp_departure_airline,
        'dp_departure_img' : dp_departure_img,
        'dp_departure_time' : dp_departure_time,
        'dp_arrival_time' : dp_arrival_time,
        'dp_duration' : dp_duration,
        'dp_departure_airport' : dp_departure_airport,
        'dp_arrival_airport' : dp_arrival_airport,
        'dp_transit' : dp_transit,
        'entry_departure_airline' : entry_departure_airline,
        'entry_departure_img' : entry_departure_img,
        'entry_departure_time' : entry_departure_time,
        'entry_arrival_time' : entry_arrival_time,
        'entry_duration' : entry_duration,
        'entry_departure_airport' : entry_departure_airport,
        'entry_arrival_airport' : entry_arrival_airport,
        'entry_transit' : entry_transit,
        'ticket_info' : ticket_info,
        'ticket_price' : ticket_price,
        'link' : link,
    }
    
    df = pd.concat([df, pd.DataFrame([one_data])], ignore_index=True)


IndexError: list index out of range

In [274]:
df

,site,dp_departure_airline,dp_departure_img,dp_departure_time,dp_arrival_time,dp_duration,dp_departure_airport,dp_arrival_airport,dp_transit,entry_departure_airline,entry_departure_img,entry_departure_time,entry_arrival_time,entry_duration,entry_departure_airport,entry_arrival_airport,entry_transit,ticket_info,ticket_price,link


In [362]:
df.to_csv('test.csv', encoding='utf-8-sig')